In [ ]:
import tensorflow as tf
import numpy as np
from model import build_graph
import cPickle
from reader import get_raw_data
import random
from reader import _word2ind

flags = tf.flags
flags.DEFINE_integer("vocab_size", 50000, "Vocab size")
flags.DEFINE_integer("emb_size", 300, "Embedding size")
flags.DEFINE_integer("cell_size", 100, "Cell size")
flags.DEFINE_integer("N", 5, "Number of test cases")
flags.DEFINE_string("cell_type", 'GRU', "Cell type")
flags.DEFINE_string("rnn_type", 'bi_dynamic', "Cell type")
flags.DEFINE_string("datafile", './_data/7000/nlu_data.pkl', "Data file (.pkl)")
flags.DEFINE_string("dictfile", './tmp/dict.pkl', "Dict file (.pkl)")
flags.DEFINE_string("modelfile", 'None', "model file")
flags.DEFINE_string("target", 'title', "What to test (title/location/data/whenst/whened/invitee)")
FLAGS = flags.FLAGS

all_data = cPickle.load(open(FLAGS.datafile))
(word2ind, ind2word) = cPickle.load(open(FLAGS.dictfile))

In [ ]:
with tf.variable_scope("Model"):
    tf_x = tf.placeholder(tf.int32, shape=(1, 60), name='x')
    tf_y = tf.placeholder(tf.int32, shape=(1, 60), name='y')
    tf_len = tf.placeholder(tf.int32, shape=(1), name='len')
    tf_mask = tf.to_float(tf.not_equal(tf_x, 0))
    g_title = build_graph(tf_x, tf_y, tf_len, tf_mask, \
                        vocab_size=FLAGS.vocab_size, \
                        emb_size=FLAGS.emb_size, \
                        cell_size=FLAGS.cell_size, \
                        cell_type=FLAGS.cell_type, \
                        rnn_type=FLAGS.rnn_type, \
                        batch_size=1, \
                        is_training=False)
    g_location = build_graph(tf_x, tf_y, tf_len, tf_mask, \
                        vocab_size=FLAGS.vocab_size, \
                        emb_size=FLAGS.emb_size, \
                        cell_size=FLAGS.cell_size, \
                        cell_type=FLAGS.cell_type, \
                        rnn_type=FLAGS.rnn_type, \
                        batch_size=1, \
                        is_training=False)
    g_invitee = build_graph(tf_x, tf_y, tf_len, tf_mask, \
                        vocab_size=FLAGS.vocab_size, \
                        emb_size=FLAGS.emb_size, \
                        cell_size=FLAGS.cell_size, \
                        cell_type=FLAGS.cell_type, \
                        rnn_type=FLAGS.rnn_type, \
                        batch_size=1, \
                        is_training=False)
    g_day = build_graph(tf_x, tf_y, tf_len, tf_mask, \
                        vocab_size=FLAGS.vocab_size, \
                        emb_size=FLAGS.emb_size, \
                        cell_size=FLAGS.cell_size, \
                        cell_type=FLAGS.cell_type, \
                        rnn_type=FLAGS.rnn_type, \
                        batch_size=1, \
                        is_training=False)
    g_whenst = build_graph(tf_x, tf_y, tf_len, tf_mask, \
                        vocab_size=FLAGS.vocab_size, \
                        emb_size=FLAGS.emb_size, \
                        cell_size=FLAGS.cell_size, \
                        cell_type=FLAGS.cell_type, \
                        rnn_type=FLAGS.rnn_type, \
                        batch_size=1, \
                        is_training=False)
    g_whened = build_graph(tf_x, tf_y, tf_len, tf_mask, \
                        vocab_size=FLAGS.vocab_size, \
                        emb_size=FLAGS.emb_size, \
                        cell_size=FLAGS.cell_size, \
                        cell_type=FLAGS.cell_type, \
                        rnn_type=FLAGS.rnn_type, \
                        batch_size=1, \
                        is_training=False)

In [ ]:
test_sents = ["i would like to go to the library at three p.m. tomorrow afternoon",
              "go swimming with alice and bob tomorrow"]

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    g_title['saver'].restore(sess, title_modelfile)
    g_location['saver'].restore(sess, location_modelfile)
    g_invitee['saver'].restore(sess, invitee_modelfile)
    g_day['saver'].restore(sess, day_modelfile)
    g_whenst['saver'].restore(sess, whenst_modelfile)
    g_whened['saver'].restore(sess, whened_modelfile)
    for _n, sent in enumerate(test_sents):
        print 
        print "test case:", _n
        print sent
        
        def process(g, text):
            tok_text = nltk.word_tokenize(text)
            _len = len(tok_text)

            x_mat = np.zeros((1, 60), dtype=np.int32)
            y_mat = np.zeros((1, 60), dtype=np.int32)
            len_mat = np.zeros((1,), dtype=np.int32)

            for i, w in enumerate(tok_text):
                x_mat[0][i] = _word2ind(w, word2ind)
            y_mat[0][:_len] = tok_y[:]
            len_mat[0] = _len

            preds = sess.run([g['preds']], feed_dict={tf_x: x_mat, tf_len: len_mat})
            #print "sent:", ' '.join(tok_text)
            #print "prediction:", preds
            pred_text = []
            for i, b in enumerate(preds):
                if i == len(tok_text):
                    break
                if b == 1:
                    pred_text.append(tok_text[i])
            return ' '.join(pred_text)
        
        pred_title = process(g_title, sent)
        pred_location = process(g_location, sent)
        pred_invitee = process(g_invitee, sent)
        pred_day = process(g_day, sent)
        pred_whenst = process(g_whenst, sent)
        pred_whened = process(g_whened, sent)
        
        print "title:", pred_title
        print "location:", pred_location
        print "invitee:", pred_invitee
        print "day:", pred_day
        print "whenst:", pred_whenst
        print "whened:", pred_whened